In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('new1_cyber_train.csv')
test_df = pd.read_csv('new1_cyber_test.csv')

In [2]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from transformers import RobertaTokenizer, TFBertForSequenceClassification  # TFBertForSequenceClassification works for Roberta too
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re

def preprocess_text(text):
  # Convert to lowercase
  text = text.lower()
  # Remove punctuation
  text = text.translate(str.maketrans("", "", string.punctuation))
  # Remove special characters
  text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  # Remove stopwords
  stop_words = set(stopwords.words('english'))
  text = ' '.join([word for word in text.split() if word not in stop_words])
  # Apply stemming
  ps = PorterStemmer()
  text = ' '.join([ps.stem(word) for word in text.split()])
  return text

train_df['text'] = train_df['text'].apply(preprocess_text)
test_df['text'] = test_df['text'].apply(preprocess_text)

# Tokenization for RoBERTa
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')  # Download RoBERTa base model

# Encoding as numbers for training dataset
encoded_train_texts = tokenizer(train_df['text'].tolist(), padding=True, truncation=True, return_tensors='tf')
X_train_padded = pad_sequences(encoded_train_texts['input_ids'], padding='post')
y_train = train_df['label'].to_numpy()

# Encoding as numbers for testing dataset
encoded_test_texts = tokenizer(test_df['text'].tolist(), padding=True, truncation=True, return_tensors='tf')
X_test_padded = pad_sequences(encoded_test_texts['input_ids'], padding='post')
y_test = test_df['label'].to_numpy()

# Building the RoBERTa model (TFBertForSequenceClassification works for Roberta too)
model = TFBertForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Compile the model (define optimizer, loss, and metric)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Training the model
history = model.fit(
  {'input_ids': X_train_padded, 'attention_mask': encoded_train_texts['attention_mask']},
  y_train,
  epochs=3,
  batch_size=32,
  validation_split=0.1
)

# Evaluating the model on the testing dataset


You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['roberta.encoder.layer.1.output.LayerNorm.weight', 'roberta.encoder.layer.4.attention.output.LayerNorm.bias', 'roberta.encoder.layer.7.output.dense.bias', 'roberta.encoder.layer.3.attention.self.value.weight', 'roberta.encoder.layer.5.attention.self.value.bias', 'roberta.encoder.layer.3.attention.self.value.bias', 'roberta.encoder.layer.11.output.dense.weight', 'roberta.encoder.layer.10.output.LayerNorm.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.encoder.layer.7.intermediate.dense.bias', 'roberta.encoder.layer.1.intermediate.dense.weight', 'roberta.encoder.layer.7.attention.self.value.bias', 'roberta.encoder.layer.10.attention.self.value.weight', 'roberta.encoder.layer.7.attention.output.dense.weight', 'roberta.encoder.layer.4.attention.self.query.weight', 'roberta.encoder.layer.8.attention.self.value.bias', 'roberta.encoder.layer.1.attention.out

Epoch 1/3

26/26 [==============================] - 778s 28s/step - loss: 0.7092 - accuracy: 0.5228 - val_loss: 0.7916 - val_accuracy: 0.4945
Epoch 2/3
26/26 [==============================] - 675s 26s/step - loss: 0.7121 - accuracy: 0.5068 - val_loss: 0.6730 - val_accuracy: 0.6703
Epoch 3/3
26/26 [==============================] - 553s 21s/step - loss: 0.6669 - accuracy: 0.5968 - val_loss: 0.6536 - val_accuracy: 0.6044


NameError: name 'y_text' is not defined

In [3]:
eval_result = model.evaluate(
  {'input_ids': X_test_padded, 'attention_mask': encoded_test_texts['attention_mask']},
  y_test
)
print(f"Test accuracy: {eval_result[1]}")


8/8 [==============================] - 32s 4s/step - loss: 0.6462 - accuracy: 0.6150
Test accuracy: 0.6150442361831665
